In [1]:
## Import All Needed Libraries

import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

from ray.tune import register_env, tune

from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
# from custom_model import *

Instructions for updating:
non-resource variables are not supported in the long term
2021-04-08 18:41:24,874	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!


In [2]:
## Reuse Wrapper for AnimalAI Environment

class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig(env_config['arena_to_train'])
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [197]:
## Setup configuration to use

conf = {
    "num_workers": 0,
    "env_config": {
        "unity_worker_id": 700,
        "arena_to_train": '../examples/configurations/curriculum/0.yml'
        },
        "model": {
            "custom_model": 'my_cnn_rnn_model',
            "custom_model_config": {},
        },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "1")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }
conf

In [198]:
## Setup and register environment
ray.shutdown()
ray.init(num_gpus=2)

ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

register_env("unity_env", lambda config: UnityEnvWrapper(config))

2021-04-09 10:03:40,209	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266


In [199]:
trainer = PPOTrainer(config=congigure, env= "unity_env")
PATH_TO_CHECKPOINT_FILE = '/home/azibit/ray_results/PPO_unity_env_2021-04-09_02-16-38lwq8v0v4/checkpoint_3500/checkpoint-3500'
# trainer.restore('/home/azibit/ray_results/PPO_unity_env_2021-04-08_16-48-575mk5ga4m/checkpoint_500/checkpoint-500')
trainer.restore(PATH_TO_CHECKPOINT_FILE)

(pid=215806) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=215806) Instructions for updating:
(pid=215806) non-resource variables are not supported in the long term
(pid=215806) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=215806) Instructions for updating:
(pid=215806) non-resource variables are not supported in the long term
(pid=215806) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future vers

RayTaskError(KeyError): [36mray::RolloutWorker.foreach_policy()[39m (pid=215809, ip=192.168.1.4)
  File "python/ray/_raylet.pyx", line 439, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 473, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 476, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 480, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 432, in ray._raylet.execute_task.function_executor
  File "/home/azibit/anaconda3/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 378, in __init__
    self.env = _validate_env(env_creator(env_context))
  File "<ipython-input-198-9949d0c48b1f>", line 11, in <lambda>
  File "<ipython-input-2-ad30242afc60>", line 7, in __init__
KeyError: 'unity_worker_id'

2021-04-11 19:21:53,886	WARNING worker.py:1107 -- The node with node id b6ff03ce948e9d132ee27a11270b5b42f08e3bdd878aae204a061012 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
(raylet) [2021-04-11 19:21:53,918 C 215778 215778] node_manager.cc:718:  Check failed: node_id != self_node_id_ Exiting because this node manager has mistakenly been marked dead by the monitor: GCS didn't receive heartbeats within timeout 30000 ms. This is likely since the machine or raylet became overloaded.
(raylet) [2021-04-11 19:21:53,920 E 215778 215778] logging.cc:415: *** Aborted at 1618186913 (unix time) try "date -d @1618186913" if you are using GNU date ***
(raylet) [2021-04-11 19:21:53,918 C 215778 215778] node_manager.cc:718:  Check failed: node_id != self_node_id_ Exiting because this node manager has mistakenly been marked dead by the monitor: GCS didn't receive heartbeats within timeout 

(raylet) [2021-04-11 19:21:54,147 E 215778 215778] logging.cc:415:     @     0x56309bb60301 boost::asio::detail::scheduler::do_run_one()
(raylet) [2021-04-11 19:21:54,151 E 215778 215778] logging.cc:415:     @     0x56309bb619a9 boost::asio::detail::scheduler::run()
(raylet) [2021-04-11 19:21:54,152 E 215778 215778] logging.cc:415:     @     0x56309bb63e97 boost::asio::io_context::run()
(raylet) [2021-04-11 19:21:54,154 E 215778 215778] logging.cc:415:     @     0x56309b3abce2 main
(raylet) [2021-04-11 19:21:54,155 E 215778 215778] logging.cc:415:     @     0x7f758abea0b3 __libc_start_main
(raylet) [2021-04-11 19:21:54,156 E 215778 215778] logging.cc:415:     @     0x56309b3c0da5 (unknown)
(raylet) [2021-04-11 19:21:54,147 E 215778 215778] logging.cc:415:     @     0x56309bb60301 boost::asio::detail::scheduler::do_run_one()
(raylet) [2021-04-11 19:21:54,151 E 215778 215778] logging.cc:415:     @     0x56309bb619a9 boost::asio::detail::scheduler::run()
(raylet) [2021-04-11 19:21:54,152 

In [192]:
trainer

In [196]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.


In [193]:
## Trying to test the trained agent on an environment

env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = 705,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('../competition_configurations/1-8-2.yml')
#             arenas_configurations = ArenaConfig('../examples/configurations/curriculum/2.yml')
)

INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


In [56]:
ray.rllib.agents.ppo.DEFAULT_CONFIG["train_batch_size"]

4000

In [194]:
# run for number of steps
state = trainer.get_policy().model.get_initial_state()


# print(obs.shape)
timesteps_to_run = 6000
timesteps_passed = 0
episode_rewards = []
while timesteps_passed < timesteps_to_run:
    obs = env.reset()
    episode_reward = 0
    done = False
    while not done:
        action = trainer.compute_action(obs, state)
        action, state = action[0], action[1]
#         print("ACTION: ", action)
        obs, reward, done, info = env.step(action)

#         print("REWARD: ", reward)
        episode_reward += reward
        timesteps_passed += 1
#     print('Episode reward:', episode_reward)
    episode_rewards.append(episode_reward)


In [195]:
print('Episodes ran:', len(episode_rewards), 'Mean episodic reward:', np.mean(episode_rewards))
j2 = [i for i in episode_rewards if i >= 0]
print("Episodes with success: ", len(j2))

Episodes ran: 29 Mean episodic reward: -0.6642643063939337
Episodes with success:  5


In [184]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.


2

In [ ]:
## Trying to automate something